In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import warnings

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from statsmodels.tsa.stattools import adfuller
from matplotlib.pyplot import figure
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.api import qqplot

warnings.filterwarnings('ignore')

In [2]:
# Read and format data
path = 'data/combined_data_new.csv'
df_all = pd.read_csv(path)
df_all = df_all.loc[df_all.Temperature.notna()]

df_all.date_time_current = pd.to_datetime(df_all.date_time_current, format = "%Y-%m-%d %H:%M:%S")
df_all.date_time_future = pd.to_datetime(df_all.date_time_future, format = "%Y-%m-%d %H:%M:%S")
df_all.date_time_current_rounded = pd.to_datetime(df_all.date_time_current_rounded, format = "%Y-%m-%d %H:%M:%S")

df_all["forecast_interval"] = df_all.date_time_future - df_all.date_time_current_rounded
df_all["forecast_error"] = df_all.total_demand - df_all.forecast_demand
df_all["forecast_error_relative"] = df_all.forecast_error/df_all.total_demand

df_all["date_time_future_month"] = df_all.date_time_future.dt.month
df_all["date_time_future_year"] = df_all.date_time_future.dt.year
df_all["date_time_future_weekday"] = df_all.date_time_future.dt.dayofweek
df_all["date_time_future_hour"] = df_all.date_time_future.dt.hour
#df_all["date_time_future_yearTime"] = df_all.date_time_future_year.apply(lambda x: pd.DateOffset(years=x-2000))

df_all["week_day_name"] = df_all.date_time_future.dt.day_name()

df_all["isSaturday"] = df_all.week_day_name.apply(lambda x: 1 if x == 'Saturday' else 0)
df_all["isSunday"] = df_all.week_day_name.apply(lambda x: 1 if x == 'Sunday' else 0)

df_all["isDecember"] = df_all.date_time_future_month.apply(lambda x: 1 if x == 12 else 0)
df_all["isJanuary"] = df_all.date_time_future_month.apply(lambda x: 1 if x == 1 else 0)
df_all["isFebruary"] = df_all.date_time_future_month.apply(lambda x: 1 if x == 2 else 0)
df_all["isNovember"] = df_all.date_time_future_month.apply(lambda x: 1 if x == 11 else 0)

## Linear Regression
Forecast interval = 12h

Check: https://realpython.com/linear-regression-in-python/

### Existing forecast model

In [3]:
delta = 24

df_lag = df_all.loc[df_all.period_id == delta].sort_values("date_time_future").reset_index(drop = True)
df_lag_temp = df_lag.copy()[["forecast_error", "forecast_error_relative", "date_time_future"]].rename({"forecast_error" : "forecast_error_24h_ago", 
                                                                                                       "forecast_error_relative":  "forecast_error_relative_24h_ago",
                                                                                                       "date_time_future": "date_time_future_24h_ago"}, axis = 1)
df_lag["date_time_current_24h_ago"] = df_lag.date_time_current - pd.DateOffset(hours = 24)
df_lag["date_time_future_24h_ago"] = df_lag.date_time_future - pd.DateOffset(hours = 24)

df_lag = df_lag.loc[df_lag.date_time_future_24h_ago >= min(df_lag.date_time_future)]
df_lag = pd.merge(df_lag, df_lag_temp, on = "date_time_future_24h_ago", how = 'left')
df_lag = df_lag.loc[df_lag.forecast_error_relative_24h_ago.notna()]

train_test_split = 0.7
split_int = int(train_test_split * len(df_lag))
df_lag_train, df_lag_test = df_lag[:split_int], df_lag[split_int:]

In [4]:
mse = mean_squared_error(df_lag_test.forecast_demand, df_lag_test.total_demand)
mape =  mean_absolute_percentage_error(df_lag_test.forecast_demand, df_lag_test.total_demand)

print(f"Existing model MSE = {round(mse)}")
print(f"Existing model MAPE = {round(100*mape,2)}%")

Existing model MSE = 55159
Existing model MAPE = 2.19%


### Linear Regression
- Forecast Error from 24h ago

In [8]:
x_columns = ["forecast_error_24h_ago"]
x = sm.add_constant(df_lag_train[x_columns])
x = sm.add_constant(x)
y = np.array(df_lag_train.forecast_error)

model = sm.OLS(y, x)
results = model.fit()
print(results.summary())

df_lag_test["lm_forecast_error_pred"] = results.predict(sm.add_constant(df_lag_test[x_columns]))
df_lag_test["lm_forecast_demand_new"] = df_lag_test.forecast_demand + df_lag_test.lm_forecast_error_pred

mse_lm1 = mean_squared_error(df_lag_test.lm_forecast_demand_new, df_lag_test.total_demand)
mape_lm1 = mean_absolute_percentage_error(df_lag_test.lm_forecast_demand_new, df_lag_test.total_demand)

print(f"\nNew model MSE = {round(mse_lm1)}")
print(f"New model MAPE = {round(100*mape_lm1,3)}%")

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.113
Model:                            OLS   Adj. R-squared:                  0.113
Method:                 Least Squares   F-statistic:                     2696.
Date:                Mon, 21 Apr 2025   Prob (F-statistic):               0.00
Time:                        18:58:17   Log-Likelihood:            -1.4233e+05
No. Observations:               21064   AIC:                         2.847e+05
Df Residuals:                   21062   BIC:                         2.847e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     10

### Linear Regression
- Forecast Error from 24h ago
- Forecast temperature

In [6]:
x_columns = ["forecast_error_24h_ago", "Temperature", "Humidity", "Wind_speed", "Rain", "isSaturday", "isSunday", "isDecember", "isJanuary", "isNovember"]
x = sm.add_constant(df_lag_train[x_columns])
x = sm.add_constant(x)
y = np.array(df_lag_train.forecast_error)

model = sm.OLS(y, x)
results = model.fit()
print(results.summary())

df_lag_test["lm2_forecast_error_pred"] = results.predict(sm.add_constant(df_lag_test[x_columns]))
df_lag_test["lm2_forecast_demand_new"] = df_lag_test.forecast_demand + df_lag_test.lm2_forecast_error_pred

mse_lm2 = mean_squared_error(df_lag_test.lm2_forecast_demand_new, df_lag_test.total_demand)
mape_lm2 = mean_absolute_percentage_error(df_lag_test.lm2_forecast_demand_new, df_lag_test.total_demand)

print(f"\nNew model MSE = {round(mse_lm2)}")
print(f"New model MAPE = {round(100*mape_lm2,2)}%")

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.124
Model:                            OLS   Adj. R-squared:                  0.124
Method:                 Least Squares   F-statistic:                     298.6
Date:                Mon, 21 Apr 2025   Prob (F-statistic):               0.00
Time:                        18:58:14   Log-Likelihood:            -1.4220e+05
No. Observations:               21064   AIC:                         2.844e+05
Df Residuals:                   21053   BIC:                         2.845e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -56

In [7]:
print(f"\nModel 1 MSE = {round(mse_lm1)}")
print(f"Model 1 MAPE = {round(100*mape_lm1,3)}%")
print(f"\nModel 2 MSE = {round(mse_lm2)}")
print(f"Model 2 MAPE = {round(100*mape_lm2,3)}%")


Model 1 MSE = 49348
Model 1 MAPE = 2.06%

Model 2 MSE = 49718
Model 2 MAPE = 2.078%


In [11]:
df_lag_test[["date_time_future", "total_demand", "forecast_demand", "lm2_forecast_demand_new"]].rename({"lm2_forecast_demand_new":"lm_prediction"}, axis = 1).to_csv("data/results_LM.csv")